In [1]:
import os
import pickle
import tarfile
import datetime
import numpy as np
import urllib.request
import sklearn.metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import keras.utils
import tensorflow.keras.utils
import shutil

2024-10-20 15:00:55.062697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 15:00:55.075423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 15:00:55.079289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 15:00:55.089420: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-20 15:00:56.275404: W tensorflow/compiler/tf2

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D, Activation
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from keras.utils import plot_model
from keras import layers, Model
import keras

In [3]:
import kagglehub
from pathlib import Path
import os
import numpy as np
import keras
import math
import tensorflow as tf
from typing import Union, Tuple
import csv
import pandas as pd
import shutil
import logging

/root/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
PATH_TO_DATA = kagglehub.dataset_download("filiprejmus/randomizedbackground100birdspecies") + '/100-bird-species'

print("Path to dataset files:", PATH_TO_DATA)

Path to dataset files: /root/.cache/kagglehub/datasets/filiprejmus/randomizedbackground100birdspecies/versions/1/100-bird-species


In [9]:
shutil.copytree(PATH_TO_DATA, 'dataset')

'dataset'

In [4]:
PATH_TO_DATA = 'dataset'

In [5]:
PATH_TO_CSV = os.getcwd() +  '/' + f'{PATH_TO_DATA}/kagle-birds.csv'

In [12]:
# Add "data set" column to table

def add_split_column_to_csv(path_to_csv: str):
  birds_data_frame = pd.read_csv(path_to_csv, index_col=0)
  file_pathes = birds_data_frame['filepaths'].to_numpy()

  splited_file_pathes = np.array([item.split('/') for item in file_pathes])
  splits = splited_file_pathes[:,3]
  split_types = np.unique(splits)

  birds_data_frame['data set'] = splits
  birds_data_frame.to_csv(path_to_csv, index=False)

In [13]:
add_split_column_to_csv(PATH_TO_CSV)

In [20]:
def fix_rel_path(path_to_csv: str):
  birds_data_frame = pd.read_csv(path_to_csv)
  file_pathes = birds_data_frame['filepaths'].to_numpy()

  relative_file_pathes = np.array(['.' + item[25:] for item in file_pathes])

  birds_data_frame.drop('filepaths', axis=1)
  birds_data_frame['filepaths'] = relative_file_pathes
  birds_data_frame.to_csv(path_to_csv, index=False)

In [21]:
fix_rel_path(PATH_TO_CSV)

['filepaths', 'labels', 'data set']


In [7]:
def image_dataset_from_directory_custom(
    directory: str,
    batch_size: int = 64,
    image_size: Tuple[int, int] = (64, 64),
    shuffle: bool = True,
    seed: int = 42,
    validation_split: float = 0.8,
    subset: Union['training', 'validation', 'both', None] = None
):
  rng = np.random.default_rng(seed)

  print('Start convert to dataset')

  sub_dir = os.listdir(directory)

  images = []
  labels = []

  for i in range(0, len(sub_dir)):
    items = os.listdir(f'{directory}/{sub_dir[i]}')

    for j in range(0, len(items)):
      image = keras.utils.load_img(f'{directory}/{sub_dir[i]}/{items[j]}', target_size=image_size)
      image_arr = keras.utils.img_to_array(image)

      images.append(image_arr)
      labels.append(i)

  print(f'{len(sub_dir)} Classes')
  print(f'{len(labels)}  Items')

  dataset = tf.data.Dataset.from_tensor_slices((images, labels))

  if not shuffle and subset is None:
    return dataset.batch(batch_size)

  dataset.shuffle(1000, seed=seed, reshuffle_each_iteration=True)

  if subset is None:
    return dataset.batch(batch_size)

  train_dataset, validation_dataset = keras.utils.split_dataset(dataset, left_size=validation_split)

  match subset:
    case 'both':
      return train_dataset.batch(batch_size), validation_dataset
    case 'training':
      return train_dataset.batch(batch_size)
    case 'validation':
      return validation_dataset

  return dataset

In [ ]:
PATH_TO_TEST = f'{PATH_TO_DATA}/test'

In [ ]:
train_dataset, validation_dataset = image_dataset_from_directory_custom(PATH_TO_TEST)

Start convert to dataset
265 Classes
1325  Items


In [ ]:
train_dataset.cardinality().numpy()

17

In [ ]:
validation_dataset.cardinality().numpy()

265

In [8]:
def load_dataset(path: str, batch_size: int, image_size: tuple[int, int], shuffle: bool, split: str) -> tuple[tf.data.Dataset, dict[int, str]]:
    temp_dir = f'{split}-temp'
    
    try:
        path_to_dir_dataset = '/'.join(path.split('/')[:-1])
        data_frame = pd.read_csv(path)
    
        temp_dir = f'{split}-temp'
    
        os.mkdir(temp_dir)
    
        filtered_data_frame = data_frame[data_frame['data set'] == split]

        index = 0
    
        for item in filtered_data_frame.iterrows():
            if index != 0:
                row = item[1]
                
                filepath = row['filepaths']
                label = row['labels']
                file_name = filepath.split('/')[-1]
    
                abs_file_path = Path(f'{path_to_dir_dataset}/{filepath}').resolve()
    
                if not os.path.exists(f'{temp_dir}/{label}'):
                    os.mkdir(f'{temp_dir}/{label}')
                    
                shutil.copy2(abs_file_path, f'{temp_dir}/{label}/{file_name}')
            
            index+=1

            if index % 100 == 0 and index != 0:
                print(f'process: {index} rows')
    
        dataset = image_dataset_from_directory_custom(temp_dir, batch_size, image_size, shuffle)
        
        values = np.unique(data_frame['labels'].to_numpy())
        keys = np.arange(values.shape[0])
        
        class_dictionary = dict(zip(keys, values))
    
        shutil.rmtree(temp_dir)
    
        return dataset, class_dictionary
    except:
        print('Errrr')
        logging.exception('')
        shutil.rmtree(temp_dir)
        return None

In [15]:
dataset, label_dict = load_dataset(PATH_TO_CSV, 64, (64, 64), False, 'test')

process: 100 rows
process: 200 rows
process: 300 rows
process: 400 rows
process: 500 rows
process: 600 rows
process: 700 rows
process: 800 rows
process: 900 rows
process: 1000 rows
process: 1100 rows
process: 1200 rows
process: 1300 rows
Start convert to dataset
265 Classes
1324  Items


In [9]:
# Model

INPUT_SHAPE = (64, 64, 3)

input = Input(shape=INPUT_SHAPE)

x = keras.layers.RandomFlip("horizontal")(input)

x = Conv2D(8, 7, activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = Conv2D(8, 5, activation='relu', padding="same")(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(265, activation="softmax")(x)

model = Model(input, output)

I0000 00:00:1729400546.530799  259398 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729400546.577742  259398 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729400546.577802  259398 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729400546.579510  259398 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729400546.579561  259398 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 64, 64, 8)      │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 8)      │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 8)      │         1,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       524,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 265)            │        17,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 544,401 (2.08 MB)

 Trainable params: 544,385 (2.08 MB)

 Non-trainable params: 16 (64.00 B)

In [12]:
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer="Adam")

In [13]:
keras.utils.set_random_seed(42)

In [14]:
v1_train = image_dataset_from_directory_custom('dataset/valid')

Start convert to dataset
265 Classes
1325  Items


In [15]:
v1_valid = image_dataset_from_directory_custom('dataset/test')

Start convert to dataset
265 Classes
1325  Items


In [21]:
v2_train, label_train = load_dataset(PATH_TO_CSV, 64, (64, 64), False, 'valid')

process: 100 rows
process: 200 rows
process: 300 rows
process: 400 rows
process: 500 rows
process: 600 rows
process: 700 rows
process: 800 rows
process: 900 rows
process: 1000 rows
process: 1100 rows
process: 1200 rows
process: 1300 rows
Start convert to dataset
265 Classes
1324  Items


In [22]:
v2_valid, label_valid = load_dataset(PATH_TO_CSV, 64, (64, 64), False, 'test')

process: 100 rows
process: 200 rows
process: 300 rows
process: 400 rows
process: 500 rows
process: 600 rows
process: 700 rows
process: 800 rows
process: 900 rows
process: 1000 rows
process: 1100 rows
process: 1200 rows
process: 1300 rows
Start convert to dataset
265 Classes
1324  Items


In [23]:
history = model.fit(v2_train, validation_data=v2_valid, epochs=1)

19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0146 - loss: 5.5658

2024-10-20 15:08:52.162780: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=0,k3=0} for conv (f32[8,8,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[44,8,64,64]{3,2,1,0}, f32[44,8,64,64]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2024-10-20 15:08:52.190837: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.984916s
Trying algorithm eng20{k2=0,k3=0} for conv (f32[8,8,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[44,8,64,64]{3,2,1,0}, f32[44,8,64,64]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_o

21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 434ms/step - accuracy: 0.0131 - loss: 5.5669 - val_accuracy: 0.0038 - val_loss: 5.5721
